# LSSTCam pointing errors
## Finding long tracking ecents

Craig Lage - 17-Aug-25

In [ ]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.tmaUtils import TMAEventMaker
from lsst.obs.lsst import LsstCam
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])
instrument = 'LSSTCam'
camera = LsstCam.getCamera()
eventMaker = TMAEventMaker()

## Finding long tracking events.

In [ ]:
startDay = 20250813
endDay = 20250816
limit = 900.0 # Find events  at least this long.
dayObs = startDay
long_events = []
while dayObs <= endDay:
    events = eventMaker.getEvents(dayObs)
    my_events = [e for e in events if (e.type.name=='TRACKING') and (e.duration > limit)]
    for event in my_events:
        long_events.append([dayObs, event.seqNum, event.duration, event.begin.utc.isot])
    dayObs = calcNextDay(dayObs)

print(len(long_events))

In [ ]:
for i,[dayObs, seqNum, duration, begin] in enumerate(long_events):
    print(i, dayObs, seqNum, duration, begin)

In [ ]:
[dayObs, seqNum, duration, begin] = long_events[27]
event = eventMaker.getEvent(dayObs, seqNum)
detector = camera['R22_S11']
bbox = detector.getBBox()
expIds = []
ras= []
decs = []
times = []
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
            where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
print(len(exposureList))
for [id,record] in exposureList:
    if record.observation_type not in ['acq', 'science']:
        continue
    if ((record.timespan.begin > event.begin) and (record.timespan.end < event.end)):
        print(record.id)
        try:
            calExp = butler.get('preliminary_visit_image', detector=94, visit=record.id, instrument=instrument)
            cWcs = calExp.getWcs()
            if cWcs == None:
                print(f"{record.id} had no cWcs.")
                continue
            calExpSkyCenter = cWcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
            ra = calExpSkyCenter.getRa().asDegrees()
            dec = calExpSkyCenter.getDec().asDegrees()
            ras.append(ra)
            decs.append(dec)
            time = (record.timespan.begin + TimeDelta(record.exposure_time / 2.0, format='sec')).utc.unix_tai
            times.append(time)
            expIds.append(record.id)
            print(ra, dec, time)
        except:
            continue

print(len(ras), len(decs), len(times))


In [ ]:
times = np.array(times)
times -= times[0]
plot_ras = np.array(ras)
plot_ras -= plot_ras[0]
plot_ras *= 3600.0
plot_decs = np.array(decs)
plot_decs -= plot_decs[0]
plot_decs *= 3600.0
fig, axs = plt.subplots(1, 2, figsize = (10, 5))
plt.subplots_adjust(wspace=0.5)
axs[0].scatter(times, plot_ras)
axs[1].scatter(times, plot_decs)
plt.suptitle(f"Tracking drift {expIds[0]} - {expIds[-1]}")
axs[0].set_xlabel("Time(sec)")
axs[0].set_ylabel("RA drift (arcsec)")
axs[1].set_xlabel("Time(sec)")
axs[1].set_ylabel("Dec drift (arcsec)")
saveFilename = f"/home/c/cslage/u/MTMount/mount_plots/Tracking_Drift_{expIds[0]}_{expIds[-1]}.png"
plt.savefig(saveFilename)

## The ones below had no hexapod motions

In [ ]:
dayObs = 20250816
seqNums = np.arange(149, 160)
input_expIds = []
for seqNum in seqNums:
    input_expIds.append(int(dayObs * 100000 + seqNum))
detector = camera['R22_S11']
bbox = detector.getBBox()
ras= []
decs = []
times = []
expIds = []
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
            where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
print(len(exposureList))
for [id,record] in exposureList:
    if record.id in input_expIds:
        print(record.id)
        try:
            calExp = butler.get('preliminary_visit_image', detector=94, visit=record.id, instrument=instrument)
            cWcs = calExp.getWcs()
            if cWcs == None:
                print(f"{record.id} had no cWcs.")
                continue
            calExpSkyCenter = cWcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
            ra = calExpSkyCenter.getRa().asDegrees()
            dec = calExpSkyCenter.getDec().asDegrees()
            ras.append(ra)
            decs.append(dec)
            time = (record.timespan.begin + TimeDelta(record.exposure_time / 2.0, format='sec')).utc.unix_tai
            times.append(time)
            expIds.append(record.id)
            print(ra, dec, time)
        except:
            continue

print(len(ras), len(decs), len(times))


In [ ]:
times = np.array(times)
times -= times[0]
plot_ras = np.array(ras)
plot_ras -= plot_ras[0]
plot_ras *= 3600.0
plot_decs = np.array(decs)
plot_decs -= plot_decs[0]
plot_decs *= 3600.0
fig, axs = plt.subplots(1, 2, figsize = (10, 5))
plt.subplots_adjust(wspace=0.5)
axs[0].scatter(times, plot_ras)
axs[1].scatter(times, plot_decs)
plt.suptitle(f"Tracking drift {expIds[0]} - {expIds[-1]}")
axs[0].set_xlabel("Time(sec)")
axs[0].set_ylabel("RA drift (arcsec)")
axs[1].set_xlabel("Time(sec)")
axs[1].set_ylabel("Dec drift (arcsec)")
saveFilename = f"/home/c/cslage/u/MTMount/mount_plots/Tracking_Drift_{expIds[0]}_{expIds[-1]}.png"
plt.savefig(saveFilename)

In [ ]:
(plot_ras[0] - plot_ras[-1]) / (times[0] - times[-1])

In [ ]:
(plot_decs[0] - plot_decs[-1]) / (times[0] - times[-1])